In [1]:
# export
!pip install tsai
from torch.nn.utils import weight_norm
from tsai.imports import *
from tsai.utils import *
from tsai.models.layers import *
from tsai.models.utils import *
import pandas as pd
import numpy as np

Defaulting to user installation because normal site-packages is not writeable


You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


/home/sasha/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# TCN IMPLEMENTATION

# This is an unofficial PyTorch implementation by Ignacio Oguiza - oguiza@gmail.com based on:

# Bai, S., Kolter, J. Z., & Koltun, V. (2018). An empirical evaluation of generic convolutional and recurrent networks for sequence modeling. arXiv preprint arXiv:1803.01271.
# Official TCN PyTorch implementation: https://github.com/locuslab/TCN


class TemporalBlock(Module):
    def __init__(self, ni, nf, ks, stride, dilation, padding, dropout=0.):
        super(TemporalBlock, self).__init__()
        self.conv1 = weight_norm(nn.Conv1d(ni,nf,ks,stride=stride,padding=padding,dilation=dilation))
        self.chomp1 = Chomp1d(padding)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout)
        self.conv2 = weight_norm(nn.Conv1d(nf,nf,ks,stride=stride,padding=padding,dilation=dilation))
        self.chomp2 = Chomp1d(padding)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(dropout)
        self.net = nn.Sequential(self.conv1, self.chomp1, self.relu1, self.dropout1, 
                                 self.conv2, self.chomp2, self.relu2, self.dropout2)
        self.downsample = nn.Conv1d(ni,nf,1) if ni != nf else None
        self.relu = nn.ReLU()
        self.init_weights()

    def init_weights(self):
        self.conv1.weight.data.normal_(0, 0.01)
        self.conv2.weight.data.normal_(0, 0.01)
        if self.downsample is not None: self.downsample.weight.data.normal_(0, 0.01)

    def forward(self, x):
        out = self.net(x)
        res = x if self.downsample is None else self.downsample(x)
        return self.relu(out + res)

def TemporalConvNet(c_in, layers, ks=2, dropout=0.):
    temp_layers = []
    for i in range(len(layers)):
        dilation_size = 2 ** i
        ni = c_in if i == 0 else layers[i-1]
        nf = layers[i]
        temp_layers += [TemporalBlock(ni, nf, ks, stride=1, dilation=dilation_size, padding=(ks-1) * dilation_size, dropout=dropout)]
    return nn.Sequential(*temp_layers)

class TCN(Module):
    def __init__(self, c_in, c_out, layers=8*[25], ks=7, conv_dropout=0., fc_dropout=0.):
        super(TCN, self).__init__()
        self.tcn = TemporalConvNet(c_in, layers, ks=ks, dropout=conv_dropout)
        self.gap = GAP1d()
        self.dropout = nn.Dropout(fc_dropout) if fc_dropout else None
        self.linear = nn.Linear(layers[-1],c_out)
        self.init_weights()

    def init_weights(self):
        self.linear.weight.data.normal_(0, 0.01)

    def forward(self, x):
        x = self.tcn(x)
        x = self.gap(x)
        if self.dropout is not None: x = self.dropout(x)
        return self.linear(x)

In [3]:
class VolForecastModule(nn.Module):
    def __init__(self, input_dim):
        super(VolForecastModule, self).__init__()
        self.input_dim = input_dim
        self.tcn = TCN(c_in=2, c_out=1, layers=5*[25], ks=5)
    
    def forward(self, x):
        return self.tcn(x)

In [4]:
def get_device():
    if torch.cuda.is_available():
        device = torch.device('cuda:0')
    else:
        device = torch.device('cpu') # don't have GPU 
    return device

def df_to_tensor(df):
    device = get_device()
    return torch.from_numpy(df.values).float().to(device)

def get_data(file_name, input_dim):
    df = pd.read_csv(file_name)
    inputs = df_to_tensor(df[['PRICE', 'SIZE']])
    outputs = df_to_tensor(df[['vol']])
    
    inputs = torch.stack([inputs[i:-input_dim+i] for i in range(input_dim)], dim = 1).transpose(1,2)
    outputs = outputs[input_dim:]
    
    return inputs, outputs

In [5]:
def train(opts, inputs, outputs, model):
    loss_fn = nn.MSELoss()
    input_dim = model.input_dim
    batch_size = opts['batch_size']
    
    train_losses = []
    
    for epoch in range(opts['nepochs']):
        inputs_batch = inputs[epoch * batch_size:(epoch + 1) * batch_size]
        outputs_batch = outputs[epoch * batch_size:(epoch + 1) * batch_size]
        loss = loss_fn(model(inputs_batch), outputs_batch)
        loss.backward()
        
        print(f"Loss: {loss}")

In [6]:
input_dim=50
model=VolForecastModule(input_dim=input_dim)
inputs, outputs = get_data('5sec_intervals_no_na.csv', input_dim)
opts = {
    "batch_size": 128,
    "nepochs": 100
}



In [7]:
train(opts, inputs, outputs, model)

RuntimeError: Input type (torch.cuda.FloatTensor) and weight type (torch.FloatTensor) should be the same